In [1]:
!pip install fastembed

   ---------------------------------------- 0.0/11.3 MB ? eta -:--:--
   ------------- -------------------------- 3.9/11.3 MB 19.5 MB/s eta 0:00:01
   ---------------------------- ----------- 8.1/11.3 MB 20.1 MB/s eta 0:00:01
   ---------------------------------------- 11.3/11.3 MB 18.6 MB/s eta 0:00:00


In [2]:
import pandas as pd
import numpy as np
from fastembed import TextEmbedding, SparseTextEmbedding
from sklearn.metrics.pairwise import cosine_similarity
import time
import warnings

warnings.filterwarnings("ignore")

In [4]:
df = pd.read_csv(r'C:\Users\dell\Desktop\MyDocs\Docs\MK\indian_faculty_dataset.csv')

print("Number of rows:", len(df))
df.head(3)

Number of rows: 15500


,Vidwan-ID,Name,Position,Department,University,Location,Profile URL,Expertise,Experience,Qualification,Honours and Awards,Highest Qualification,Has Awards,Start Year,Years of Experience
0,556330,Ms Dixita Kagathara,Assistant Professor,Department of Computer Science and Engineering,Darshan University,Gujarat,https://darshan.irins.org/profile/556330,Computer Science Software Engineering,2021 - Present; Ass...,2013; MTech; Gujarat University,NaN,MTech,0,2021.0,3.0
1,556331,Mr Firoz Sherasiya,Assistant Professor,Department of Computer Science and Engineering,Darshan University,Gujarat,https://darshan.irins.org/profile/556331,Computer Science Software Engineering,2022 - Present; Ass...,2015; MTech; Gujarat Technological University,NaN,MTech,0,2022.0,2.0
2,556358,Mr Bhavin Kanani,Assistant Professor,Department of Electrical Engineering,Darshan University,Gujarat,https://darshan.irins.org/profile/556358,Electrical and Electronic Engineering,2021 - Present; Ass...,2008; PhD; Gujarat Technological University,NaN,PhD,0,2021.0,3.0


In [5]:
def combine_text_fields(row):
    """Combine relevant columns into a single string for embedding."""
    fields = [
        str(row.get("Name", "")),
        str(row.get("Position", "")),
        str(row.get("Department", "")),
        str(row.get("University", "")),
        str(row.get("Expertise", "")),
        str(row.get("Honours and Awards", "")),
        str(row.get("Location", "")),	
    ]
    return " | ".join([f for f in fields if f])

df["combined_text"] = df.apply(combine_text_fields, axis=1)
df["combined_text"].head()

0    Ms Dixita Kagathara | Assistant Professor | De...
1    Mr Firoz Sherasiya | Assistant Professor | Dep...
2    Mr Bhavin Kanani | Assistant Professor | Depar...
3    Ms Vrunda Amrutiya | Assistant Professor | Dep...
4    Mr Raj Mehta | Assistant Professor | Departmen...
Name: combined_text, dtype: object

In [6]:
df = df.sample(50, random_state=890).reset_index(drop=True)

In [7]:
dense_model = TextEmbedding(model_name="BAAI/bge-small-en-v1.5")
sparse_model = SparseTextEmbedding(model_name="prithivida/Splade_PP_en_v1")

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

config.json:   0%|          | 0.00/706 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

model_optimized.onnx:   0%|          | 0.00/66.5M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.24k [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.38k [00:00<?, ?B/s]

model.onnx:   0%|          | 0.00/532M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/755 [00:00<?, ?B/s]

2025-01-16 21:32:37.402 | ERROR    | fastembed.common.model_management:download_model:274 - Could not download model from HuggingFace: [WinError 1314] A required privilege is not held by the client: '..\\..\\blobs\\9bbecc17cabbcbd3112c14d6982b51403b264bfa' -> 'C:\\Users\\dell\\AppData\\Local\\Temp\\fastembed_cache\\models--Qdrant--SPLADE_PP_en_v1\\snapshots\\efcd182bc7eb351e81a9445752d4388c2bab500b\\special_tokens_map.json' Falling back to other sources.
2025-01-16 21:32:37.402 | ERROR    | fastembed.common.model_management:download_model:295 - Could not download model from either source, sleeping for 3.0 seconds, 2 retries left.


In [8]:
dense_input_docs = df["combined_text"].tolist()
dense_vectors = list(dense_model.passage_embed(dense_input_docs))

# Sparse Embeddings (using prithivida/Splade_PP_en_v1)
sparse_input_docs = df["combined_text"].tolist()
sparse_vectors = list(sparse_model.passage_embed(sparse_input_docs))

df["dense_embedding"] = dense_vectors
df["sparse_embedding"] = sparse_vectors

print(f"Dense Embedding Dim: {len(df['dense_embedding'][0])}")
print("Sparse Embedding Example (indices/values):", df["sparse_embedding"][0])

Dense Embedding Dim: 384
Sparse Embedding Example (indices/values): SparseEmbedding(values=array([0.57314658, 0.06666028, 0.14102256, 0.82257289, 0.981305  ,
       0.01780456, 0.96968335, 0.83644873, 0.89304149, 0.90372348,
       0.49755338, 0.21594474, 0.20043802, 1.19442868, 0.21652034,
       1.00194573, 0.20309684, 1.33174181, 0.82499838, 1.46889472,
       1.10186934, 0.36939579, 0.3927218 , 0.26545632, 1.3722887 ,
       0.09362   , 0.27352765, 1.59535003, 0.99887669, 0.35719705,
       0.05526805, 0.47717029, 0.47269845, 0.40670124, 0.05592263,
       0.12073908, 0.20867622, 2.6108191 , 0.51808959, 0.59664899,
       0.46154112, 1.54633021, 0.11469325, 0.34793094, 0.06080704,
       0.15513813, 0.32537818, 1.61819375, 0.08000456, 0.18148032,
       0.08159585, 0.81181163, 1.56031966, 0.73602504, 0.94190085,
       0.51210779, 0.47059438, 2.684659  ]), indices=array([ 1055,  2010,  2016,  2040,  2078,  2082,  2118,  2267,  2533,
        2634,  2671,  2796,  2820,  2852,  2902, 

In [9]:
def hybrid_search(query, df, dense_model, sparse_model, alpha=1.0, beta=1.0, top_k=5):
    """
    Perform Hybrid Search over the provided dataframe:
      1) Generate dense and sparse embeddings for 'query'
      2) Calculate cos similarity for dense vectors
      3) Calculate dot or cos for sparse vectors
      4) Combine the two scores via alpha, beta
      5) Return top_k results
    """
    # Generate query embeddings
    query_dense = list(dense_model.embed([query]))[0]
    query_sparse = list(sparse_model.embed([query]))[0]


    query_dense_arr = np.array(query_dense).reshape(1, -1)
    query_sparse_dict = dict(zip(query_sparse.indices, query_sparse.values))

    results = []
    for i, row in df.iterrows():
        doc_dense = np.array(row["dense_embedding"]).reshape(1, -1)
        doc_sparse = dict(zip(row["sparse_embedding"].indices, row["sparse_embedding"].values))

        # Compute Dense similarity
        dense_score = cosine_similarity(query_dense_arr, doc_dense)[0][0]

        # Compute Sparse similarity (dot-product)
        common_indices = set(query_sparse_dict.keys()).intersection(set(doc_sparse.keys()))
        sparse_score = sum(query_sparse_dict[idx] * doc_sparse[idx] for idx in common_indices)

        # Combine Scores 
        final_score = alpha * dense_score + beta * sparse_score
        results.append((i, final_score))

    # Sort by final hybrid score in descending order
    results.sort(key=lambda x: x[1], reverse=True)

    top_indices = [t[0] for t in results[:top_k]]
    top_scores  = [t[1] for t in results[:top_k]]

    return top_indices, top_scores

In [10]:
def perform_query_and_display(query, df, dense_model, sparse_model, alpha=1.0, beta=1.0, top_k=5):
    """
    Perform Hybrid Search and display results for a given query.
    
    Parameters:
    - query: The query string to search for.
    - df: The dataframe containing the data.
    - dense_model: Pre-trained dense embedding model.
    - sparse_model: Pre-trained sparse embedding model.
    - alpha: Weight for the dense similarity score.
    - beta: Weight for the sparse similarity score.
    - top_k: Number of top results to display.
    """
    # Measure the query time
    start_time = time.time()
    
    # Perform hybrid search
    top_indices, top_scores = hybrid_search(
        query=query,
        df=df,
        dense_model=dense_model,
        sparse_model=sparse_model,
        alpha=alpha,
        beta=beta,
        top_k=top_k
    )
    
    end_time = time.time()
    query_time = end_time - start_time

    # Display results
    print("===============================================================")
    print(f"\033[92m\033[1mResults for Query:\033[0m \033[94m{query}\033[0m")
    print("===============================================================")
    print(f"\033[92mQuery Time:\033[0m {query_time:.4f} seconds")
    print("===============================================================")

    for rank, (idx, score) in enumerate(zip(top_indices, top_scores), start=1):
        row = df.iloc[idx]
        print(f"\033[93mRank #{rank}\033[0m | \033[92mHybrid Score:\033[0m {score:.4f}")
        print(f"Name: {row['Name']}")
        print(f"Position: {row['Position']}")
        print(f"Department/Expertise: {row['Department']}, {row['Expertise']}")
        print(f"University: {row['University']}")
        print(f"Location: {row['Location']}")
        print("------")

In [11]:
perform_query_and_display("Indian Institute of Technology Kharagpur",
                          df, dense_model, sparse_model,
                          alpha=1.0, beta=0, top_k=3)

Results for Query: Indian Institute of Technology Kharagpur
Query Time: 0.2391 seconds
Rank #1 | Hybrid Score: 0.6826
Name: Dr S Santhosh Baboo
Position: Principal
Department/Expertise: Department of MCA, 7
University: Dwaraka Doss Goverdhan Doss Vaishnav College
Location: Tamil Nadu
------
Rank #2 | Hybrid Score: 0.6520
Name: Dr Karuppaiah P
Position: Professor
Department/Expertise: Department of Horticulture, Horticulture
University: Annamalai University
Location: Tamil Nadu
------
Rank #3 | Hybrid Score: 0.6380
Name: Dr Rajini G
Position: Professor
Department/Expertise: Management Studies, Management
University: Vels Institute of Science, Technology and Advanced Studies (VISTAS)
Location: Tamil Nadu
------


In [12]:
perform_query_and_display("Indian Institute of Technology Kharagpur ",
                          df, dense_model, sparse_model,
                          alpha=0, beta=1.0, top_k=3)

Results for Query: Indian Institute of Technology Kharagpur 
Query Time: 0.1878 seconds
Rank #1 | Hybrid Score: 20.7817
Name: Dr TRILOK SINGH
Position: Assistant Professor (Grade-I)
Department/Expertise: Department of Materials Science, Materials Science Multidisciplinary
University: Indian Institute of Technology Kharagpur
Location: West Bengal
------
Rank #2 | Hybrid Score: 20.1012
Name: Prof Kamal Lochan Panigrahi
Position: Professor
Department/Expertise: Department of Physics, Multidisciplinary Physics
University: Indian Institute of Technology Kharagpur
Location: West Bengal
------
Rank #3 | Hybrid Score: 19.9671
Name: Prof Debjani Chakraborty
Position: Professor
Department/Expertise: Department of Mathematics, Mathematics
University: Indian Institute of Technology Kharagpur
Location: West Bengal
------


In [13]:
perform_query_and_display("Indian Institute of Technology Kharagpur",
                          df, dense_model, sparse_model,
                          alpha=0.75, beta=0.25, top_k=3)

Results for Query: Indian Institute of Technology Kharagpur
Query Time: 0.2451 seconds
Rank #1 | Hybrid Score: 5.6668
Name: Dr TRILOK SINGH
Position: Assistant Professor (Grade-I)
Department/Expertise: Department of Materials Science, Materials Science Multidisciplinary
University: Indian Institute of Technology Kharagpur
Location: West Bengal
------
Rank #2 | Hybrid Score: 5.4409
Name: Prof Kamal Lochan Panigrahi
Position: Professor
Department/Expertise: Department of Physics, Multidisciplinary Physics
University: Indian Institute of Technology Kharagpur
Location: West Bengal
------
Rank #3 | Hybrid Score: 5.4278
Name: Prof Debjani Chakraborty
Position: Professor
Department/Expertise: Department of Mathematics, Mathematics
University: Indian Institute of Technology Kharagpur
Location: West Bengal
------


In [14]:
# Vector Search
perform_query_and_display("An expert in a the branch of science that deals with the structure of things around us and how the fundamental constituents of the universe interact.",
                          df, dense_model, sparse_model,
                          alpha=1.0, beta=0, top_k=3)

Results for Query: An expert in a the branch of science that deals with the structure of things around us and how the fundamental constituents of the universe interact.
Query Time: 0.2511 seconds
Rank #1 | Hybrid Score: 0.5603
Name: Prof Kamal Lochan Panigrahi
Position: Professor
Department/Expertise: Department of Physics, Multidisciplinary Physics
University: Indian Institute of Technology Kharagpur
Location: West Bengal
------
Rank #2 | Hybrid Score: 0.5528
Name: Prof Souvik Chakraborty
Position: Assistant Professor
Department/Expertise: Department of Applied Mechanics, Applied Mathematics
University: Indian Institute of Technology Delhi
Location: nan
------
Rank #3 | Hybrid Score: 0.5474
Name: Dr Navdeep Goyal
Position: Dean
Department/Expertise: Department of Physics, Condensed Matter Physics
University: Panjab University
Location: Chandigarh
------


In [15]:
# Keyword Search
perform_query_and_display("An expert in a the branch of science that deals with the structure of things around us and how the fundamental constituents of the universe interact.",
                          df, dense_model, sparse_model,
                          alpha=0, beta=1.0, top_k=3)

Results for Query: An expert in a the branch of science that deals with the structure of things around us and how the fundamental constituents of the universe interact.
Query Time: 0.2077 seconds
Rank #1 | Hybrid Score: 3.8383
Name: Dr Jessy Jose
Position: Assistant Professor
Department/Expertise: Physics, Astrophysics
University: Indian Institute of Science Education and Research, Tirupati
Location: Andhra Pradesh
------
Rank #2 | Hybrid Score: 3.4995
Name: Dr Rajini G
Position: Professor
Department/Expertise: Management Studies, Management
University: Vels Institute of Science, Technology and Advanced Studies (VISTAS)
Location: Tamil Nadu
------
Rank #3 | Hybrid Score: 3.3762
Name: Dr Utpal S Joshi
Position: Professor
Department/Expertise: Department of Physics, Electronics and Space Science, Condensed Matter Physics
University: Gujarat University
Location: Gujarat
------


In [16]:
# Hybrid Search
perform_query_and_display("An expert in a the branch of science that deals with the structure of things around us and how the fundamental constituents of the universe interact.",
                          df, dense_model, sparse_model,
                          alpha=0.75, beta=0.25, top_k=3)

Results for Query: An expert in a the branch of science that deals with the structure of things around us and how the fundamental constituents of the universe interact.
Query Time: 0.1904 seconds
Rank #1 | Hybrid Score: 1.3655
Name: Dr Jessy Jose
Position: Assistant Professor
Department/Expertise: Physics, Astrophysics
University: Indian Institute of Science Education and Research, Tirupati
Location: Andhra Pradesh
------
Rank #2 | Hybrid Score: 1.2649
Name: Dr Rajini G
Position: Professor
Department/Expertise: Management Studies, Management
University: Vels Institute of Science, Technology and Advanced Studies (VISTAS)
Location: Tamil Nadu
------
Rank #3 | Hybrid Score: 1.2354
Name: Dr Utpal S Joshi
Position: Professor
Department/Expertise: Department of Physics, Electronics and Space Science, Condensed Matter Physics
University: Gujarat University
Location: Gujarat
------


In [17]:
# Vector Search
perform_query_and_display("An expert in Chemistry based in Tamil Nadu",
                          df, dense_model, sparse_model,
                          alpha=1.0, beta=0.0, top_k=3)

Results for Query: An expert in Chemistry based in Tamil Nadu
Query Time: 0.1451 seconds
Rank #1 | Hybrid Score: 0.7664
Name: Dr Srinivasan Krishnan
Position: Associate Professor
Department/Expertise: Department of Chemistry, Inorganic and Nuclear Chemistry
University: Thiruvalluvar University
Location: Tamil Nadu
------
Rank #2 | Hybrid Score: 0.7418
Name: Dr Manoj Kumar Bharty
Position: Assistant Professor
Department/Expertise: Department of Chemistry, Multidisciplinary Chemistry
University: Banaras Hindu University
Location: Uttar Pradesh
------
Rank #3 | Hybrid Score: 0.7412
Name: Prof Debashis Chakraborty
Position: Professor
Department/Expertise: Department of Chemistry, Organic Chemistry
University: Indian Institute of Technology Madras
Location: Tamil Nadu
------


In [18]:
# Keyword Search
perform_query_and_display("An expert in Chemistry based in Tamil Nadu",
                          df, dense_model, sparse_model,
                          alpha=0.0, beta=1.0, top_k=3)

Results for Query: An expert in Chemistry based in Tamil Nadu
Query Time: 0.1660 seconds
Rank #1 | Hybrid Score: 18.2916
Name: Dr Srinivasan Krishnan
Position: Associate Professor
Department/Expertise: Department of Chemistry, Inorganic and Nuclear Chemistry
University: Thiruvalluvar University
Location: Tamil Nadu
------
Rank #2 | Hybrid Score: 17.0644
Name: Prof Debashis Chakraborty
Position: Professor
Department/Expertise: Department of Chemistry, Organic Chemistry
University: Indian Institute of Technology Madras
Location: Tamil Nadu
------
Rank #3 | Hybrid Score: 14.2652
Name: Dr Geethanjali Purushothaman
Position: Associate Professor
Department/Expertise: School of Electrical Engineering, Biomedical Engineering
University: Vellore Institute of Technology
Location: Tamil Nadu
------


In [19]:
# Hybrid Search
perform_query_and_display("An expert in Chemistry based in Tamil Nadu",
                          df, dense_model, sparse_model,
                          alpha=0.75, beta=0.25, top_k=3)

Results for Query: An expert in Chemistry based in Tamil Nadu
Query Time: 0.1334 seconds
Rank #1 | Hybrid Score: 5.1477
Name: Dr Srinivasan Krishnan
Position: Associate Professor
Department/Expertise: Department of Chemistry, Inorganic and Nuclear Chemistry
University: Thiruvalluvar University
Location: Tamil Nadu
------
Rank #2 | Hybrid Score: 4.8220
Name: Prof Debashis Chakraborty
Position: Professor
Department/Expertise: Department of Chemistry, Organic Chemistry
University: Indian Institute of Technology Madras
Location: Tamil Nadu
------
Rank #3 | Hybrid Score: 4.0418
Name: Dr Balasundaram O N
Position: Associate Professor
Department/Expertise: Department of Physics, Materials Science Coatings and Films
University: PSG College of Arts and Science
Location: Tamil Nadu
------
